In [1]:
import sys
sys.path.append('../')

from model_utils import load_model

MODEL_NAME = 'vgg16_imagenet'
# MODEL_NAME = 'resnet50_imagenet'
# MODEL_NAME = 'resnet18_imagenet'

# MODEL_NAME = 'faster_rcnn_vgg16
# MODEL_NAME = 'faster_rcnn_resnet50'


# # Uncomment if MODEL_NAME = 'faster_rcnn_resnet50'
# sys.path.append('/workspace/home/jgusak/maxvol_objects/facebook_frcnn/')
# import maskrcnn_benchmark

model = load_model(MODEL_NAME)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

### Get  all  layers

Function  **get_layer_names()** returns names of model layers (convolutional and fully connected) and boolean mask for convolutional layers. 

In [2]:
from model_utils import get_layer_names

layer_names, conv_layer_mask = get_layer_names(model)

fc_layer_mask = (1 - conv_layer_mask).astype(bool)

print(layer_names[conv_layer_mask])
print(layer_names[fc_layer_mask])

['features.0' 'features.2' 'features.5' 'features.7' 'features.10'
 'features.12' 'features.14' 'features.17' 'features.19' 'features.21'
 'features.24' 'features.26' 'features.28']
['classifier.0' 'classifier.3' 'classifier.6']


In [4]:
# auxiliary function
import numpy as np
def split_resnet_layers_by_blocks(lnames):
    starts = ['body.stem.conv1'] + ['body.layer{}'.format(i) for i in range(1,5)]

    start_idx = 0
    blocks_idxs = []
    layer_names_by_blocks = []

    for s in starts:
        curr_block =  [l for l in lnames if l.startswith(s)]
        layer_names_by_blocks.append(curr_block)

        blocks_idxs.append(np.arange(start_idx, start_idx+len(curr_block)))
        start_idx += len(curr_block)

    return blocks_idxs

### Compress selected layers

For **convolutional** layers
- Set **decomposition**: 'tucker2', 'cp3' or 'cp4'
- Set  decomposition **ranks** for convolutional layers (namely, ranks we use to decompose convolutional weight tensors). 
  - In Tucker2 case, for one layer 
      - If **rank = None**, the layer won't be decomposed.
      - Elif **rank = 0**, then  VBMF method with **vbmf_weaken_factor**  will be used to select (rank_cout, rank_cin).
      - Elif **rank = (-scalar) < 0**, then values (rank_cout, rank_cin) will be choosen as maximal values which allow **(sacalar x) layer parameter reduction**.
      - Else **rank = tuple** and determines absolute ranks values (rank_cout, rank_cin)
  - In CP case, rank for one layer is a scalar
      - If **rank = None**, the layer won't be decomposed.
      - Elif **rank = (-scalar) < 0** then value for rank will be choosen as maximal rank which allows **(sacalar x) layer parameter reduction**.
      - Else **rank = scalar > 0** and determines absolute rank value.
      
For **fully connected** layers
- Set **decomposition** = 'svd'
- Set decomposition for linear layers (namely, ranks we use to factorize weight matrices)
    - In SVD case, rank for one layer is a scalar
      - If **rank = None**, the layer won't be decomposed.
      - Elif **rank = 0**, then  VBMF method with **vbmf_weaken_factor**  will be used to select rank.
      - Elif **rank = (-scalar) < 0** then value for rank will be choosen as maximal rank which allows **(sacalar x) layer parameter reduction**.
      - Else **rank = scalar > 0** and determines absolute rank value.

In [9]:
from tensor_compression import get_compressed_model
import copy
import numpy as np

# decomposition_conv = 'cp4'
decomposition_conv = 'cp3'
# decomposition_conv = 'tucker2'

decomposition_fc = 'svd'

# RANK_SELECTION = 'vbmf'
RANK_SELECTION = 'nx'
# RANK_SELECTION = 'custom'

if RANK_SELECTION == 'vbmf':
    WEAKEN_FACTOR = 1.
    X_FACTOR = 0
    rank_selection_suffix = "/wf:{}".format(WEAKEN_FACTOR)
elif RANK_SELECTION == 'nx':
    WEAKEN_FACTOR = None  
    X_FACTOR = 20
    rank_selection_suffix = "/{}x".format(X_FACTOR)
    
    
if MODEL_NAME == 'vgg16_imagenet':
    ranks_conv = [None] + [-X_FACTOR]*(len(layer_names[conv_layer_mask])-1)
    ranks_fc = [-X_FACTOR]*(len(layer_names[fc_layer_mask]))
elif MODEL_NAME == 'resnet50_imagenet':
    ranks_conv = [None if not name.endswith('conv2') else -X_FACTOR
                  for name in layer_names[conv_layer_mask]]
    ranks_fc = [-X_FACTOR]*(len(layer_names[fc_layer_mask]))
elif MODEL_NAME == 'resnet18_imagenet':
    ranks_conv = [None if name == 'conv1' or not (name.endswith('conv2') or
                                                  name.endswith('conv1')) else -X_FACTOR
              for name in layer_names[conv_layer_mask]]
    ranks_fc = [-X_FACTOR]*(len(layer_names[fc_layer_mask]))
elif MODEL_NAME ==  'faster_rcnn_resnet50':
    ranks_conv = [None if not name.endswith('body.conv2') else -X_FACTOR
                  for name in layer_names[conv_layer_mask]]
    ranks_fc = [-X_FACTOR]*(len(layer_names[fc_layer_mask]))
    
    

ranks = np.array([None]*len(layer_names))
ranks[conv_layer_mask] = ranks_conv
ranks[fc_layer_mask] = ranks_fc

decompositions = np.array([None]*len(layer_names))
decompositions[conv_layer_mask] = decomposition_conv
decompositions[fc_layer_mask] = decomposition_fc

CONV_SPLIT = 3
FC_SPLIT = 1
n_layers = len(layer_names)

RESNET_SPLIT = False
if MODEL_NAME in ['resnet50_imagenet', 'resnet18_imagenet',  'faster_rcnn_resnet50'] and RESNET_SPLIT:
    split_tuples = split_resnet_layers_by_blocks(layer_names[conv_layer_mask])[::-1]
else:
    split_tuples = np.array_split(np.arange(n_layers)[conv_layer_mask], CONV_SPLIT)[::-1]
split_tuples.append(np.array_split(np.arange(n_layers)[fc_layer_mask], FC_SPLIT))

In [10]:
split_tuples

[array([ 9, 10, 11, 12]),
 array([5, 6, 7, 8]),
 array([0, 1, 2, 3, 4]),
 [array([13, 14, 15])]]

In [ ]:
compressed_model = copy.deepcopy(model)
for tupl in split_tuples:
    lname, rank, decomposition = layer_names[tupl], ranks[tupl], decompositions[tupl]
    print(lname, rank)
    compressed_model = get_compressed_model(compressed_model,
                                            ranks=rank,
                                            layer_names=lname,
                                            decompositions = decomposition,
                                            vbmf_weaken_factor=WEAKEN_FACTOR)

['features.21' 'features.24' 'features.26' 'features.28'] [-20 -20 -20 -20]
Decompose layer features.21


In [12]:
compressed_model

Sequential(
  (body): ResNet(
    (stem): StemWithFixedBatchNorm(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
    )
    (layer1): Sequential(
      (0): BottleneckWithFixedBatchNorm(
        (downsample): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): FrozenBatchNorm2d()
        )
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): FrozenBatchNorm2d()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): FrozenBatchNorm2d()
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): FrozenBatchNorm2d()
      )
      (1): BottleneckWithFixedBatchNorm(
        (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): FrozenBatchNorm2d()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stri

# Count parameters

In [13]:
from collections import defaultdict

def count_params(model):
    n_params = 0
    
    for name, param in model.named_parameters():
        n_params += param.numel()
    return n_params

In [14]:
params_count_dict_m = count_params(model)
params_count_dict_cm = count_params(compressed_model)

params_count_dict_m / params_count_dict_cm

1.0

In [15]:
split_tuples

[array([29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]),
 array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 [array([], dtype=int64)]]

# Compute FLOPS

In [17]:
import torch
import numpy as np
from model_utils import  compute_model_flops, \
get_input_shapes_vgg, get_input_shapes_resnet, \
get_input_shapes_faster_rcnn_vgg, get_input_shapes_faster_rcnn_resnet50

device = 'cuda'
x = torch.randn(1, 3, 224, 224).to(device)

if MODEL_NAME == 'vgg16_imagenet':
    input_shapes_dict = get_input_shapes_vgg(model, x)
    input_shapes_dict_cm = get_input_shapes_vgg(compressed_model, x)
elif MODEL_NAME in 'resnet50_imagenet':
    input_shapes_dict = get_input_shapes_resnet(model, x, resnet_id = 50)
    input_shapes_dict_cm = get_input_shapes_resnet(compressed_model, x, resnet_id = 50)
elif MODEL_NAME == 'resnet18_imagenet':
    input_shapes_dict = get_input_shapes_resnet(model, x, resnet_id = 18)
    input_shapes_dict_cm = get_input_shapes_resnet(compressed_model, x, resnet_id = 18)
elif MODEL_NAME == 'faster_rcnn_vgg16':
    input_shapes_dict = get_input_shapes_faster_rcnn_vgg(model, x)
elif MODEL_NAME == 'faster_rcnn_resnet50':
    input_shapes_dict = get_input_shapes_faster_rcnn_resnet50(model, x)


In [18]:
input_shapes_dict

{'body.stem.conv1': torch.Size([1, 3, 224, 224]),
 'body.layer1.0.conv1': torch.Size([1, 64, 56, 56]),
 'body.layer1.0.conv2': torch.Size([1, 64, 56, 56]),
 'body.layer1.0.conv3': torch.Size([1, 64, 56, 56]),
 'body.layer1.0.downsample.0': torch.Size([1, 64, 56, 56]),
 'body.layer1.1.conv1': torch.Size([1, 256, 56, 56]),
 'body.layer1.1.conv2': torch.Size([1, 64, 56, 56]),
 'body.layer1.1.conv3': torch.Size([1, 64, 56, 56]),
 'body.layer1.2.conv1': torch.Size([1, 256, 56, 56]),
 'body.layer1.2.conv2': torch.Size([1, 64, 56, 56]),
 'body.layer1.2.conv3': torch.Size([1, 64, 56, 56]),
 'body.layer2.0.conv1': torch.Size([1, 256, 56, 56]),
 'body.layer2.0.conv2': torch.Size([1, 128, 28, 28]),
 'body.layer2.0.conv3': torch.Size([1, 128, 28, 28]),
 'body.layer2.0.downsample.0': torch.Size([1, 256, 56, 56]),
 'body.layer2.1.conv1': torch.Size([1, 512, 28, 28]),
 'body.layer2.1.conv2': torch.Size([1, 128, 28, 28]),
 'body.layer2.1.conv3': torch.Size([1, 128, 28, 28]),
 'body.layer2.2.conv1': to

In [9]:
flops_m = compute_model_flops(model, input_shapes_dict)
flops_cm = compute_model_flops(compressed_model, input_shapes_dict_cm)

for flops, model_title in zip([flops_m, flops_cm],
                              ['Initial model', 'Compressed model']):
    print('{}, conv: {}({}), fc: {}({})'.format(model_title,
                                                flops['conv'],
                                                np.round(flops['conv']/sum(flops.values()), decimals = 2),
                                                flops['fc'],
                                                np.round(flops['fc']/sum(flops.values()), decimals = 2)))

print('\n FLOPS speed-up, conv: {}, total: {}'.format(flops_m['conv']/flops_cm['conv'],
                                                   sum(flops_m.values())/sum(flops_cm.values())
                                                  ))
if flops_cm['fc'] > 0:
    print(' FLOPS speed-up, fc: {}, total: {}'.format(flops_m['fc']/flops_cm['fc'],
                                                      sum(flops_m.values())/sum(flops_cm.values())
                                                     ))

Initial model, conv: 15346630656(0.99), fc: 123633664(0.01)
Compressed model, conv: 15346630656(0.99), fc: 123633664(0.01)

 FLOPS speed-up, conv: 1.0, total: 1.0
 FLOPS speed-up, fc: 1.0, total: 1.0


In [10]:
compute_model_flops(compressed_model, input_shapes_dict_cm, verbose = True)


features.0 0.01
features.2 0.12
features.5 0.06
features.7 0.12
features.10 0.06
features.12 0.12
features.14 0.12
features.17 0.06
features.19 0.12
features.21 0.12
features.24 0.03
features.26 0.03
features.28 0.03
classifier.0 0.01
classifier.3 0.0
classifier.6 0.0



{'conv': 15346630656, 'fc': 123633664}